# Step 1: Import Libraries
We begin by importing the essential libraries for working with neuroimaging data (nibabel), numerical operations (numpy), file handling (os), and interactive plotting (plotly).

In [ ]:
# File handling and numerical operations
import os
import numpy as np

# Neuroimaging data handling
import nibabel as nib

# Plotly for interactive 3D visualizations
import plotly.graph_objects as go


# Step 2: Define Dataset Path and Explore Contents
We'll define the path to the BraTS2020 validation data, check that it exists, and list the patient folders to make sure we're reading the structure correctly.

In [ ]:
# Define the path to the validation dataset
base_path = "../data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData"

# Check if the path exists
if not os.path.exists(base_path):
    raise FileNotFoundError(f"Path does not exist: {base_path}")

# List all patient folders inside the validation set
patient_folders = [f for f in os.listdir(base_path) if f.startswith("BraTS20_Validation_")]

# Show how many we found and list the first few as a sanity check
print(f"🧠 Found {len(patient_folders)} patient folders.")
print("📄 First 5 folders:", patient_folders[:5])


# Step 3: Load a FLAIR MRI Scan for One Patient
We'll pick the first patient, locate the corresponding FLAIR .nii file, and load it using nibabel.

In [ ]:
# Pick the first patient folder
sample_id = patient_folders[0]
sample_path = os.path.join(base_path, sample_id)

# Construct full path to the FLAIR scan file
flair_file = os.path.join(sample_path, f"{sample_id}_flair.nii")

# Check if file exists
if not os.path.exists(flair_file):
    raise FileNotFoundError(f"FLAIR file not found: {flair_file}")

# Load the MRI scan
img = nib.load(flair_file)
img_data = img.get_fdata()

# Show image shape and value range for inspection
print(f"📐 Shape: {img_data.shape}")
print(f"🔢 Min: {np.min(img_data):.2f}, Max: {np.max(img_data):.2f}")


# Step 4: Normalize and Threshold the Image Data
To make the visualization meaningful, we’ll:
- Normalize intensity values to a 0–1 scale.
- Apply a threshold to highlight relevant brain tissue and ignore noise.

In [ ]:
# Normalize image data to 0–1 range
img_data_norm = (img_data - np.min(img_data)) / (np.max(img_data) - np.min(img_data))

# Apply a threshold to ignore low-intensity voxels
threshold = 0.1
img_masked = np.where(img_data_norm > threshold, img_data_norm, 0)

# Subsample the volume for performance
img_sub = img_masked[::2, ::2, ::2]

# Diagnostic output
print(f"🧪 Subsampled shape: {img_sub.shape}")
print(f"🔍 Unique values in img_sub: {np.unique(img_sub)[:10]}")
print(f"🔢 Non-zero voxels after threshold: {np.count_nonzero(img_sub)}")


# Step 5: Plot 3D Brain Structure with Plotly
We’ll use Scatter3d from Plotly to render an interactive 3D view of the non-zero voxels.

In [ ]:
# Create grid coordinates
x = np.arange(img_sub.shape[0])
y = np.arange(img_sub.shape[1])
z = np.arange(img_sub.shape[2])

# Generate full coordinate grid
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# Flatten all for Plotly
fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=img_sub.flatten(),
    opacity=0.1,
    surface_count=15,
    isomin=threshold,
    isomax=1.0,
    colorscale='Plasma',
    caps=dict(x_show=False, y_show=False, z_show=False)
))

fig.update_layout(
    title=f"{sample_id} - FLAIR MRI (Normalized Volume Rendering)",
    width=800,
    height=800,
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()
